# Setup

In [1]:
pip install -r "https://raw.githubusercontent.com/MDA-Panama/Project/main/requirements.txt"

You should consider upgrading via the 'pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
from pandas.plotting import parallel_coordinates
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns; sns.set()
import warnings
import statistics as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly as py
from plotly.offline import download_plotlyjs,init_notebook_mode, plot,iplot
import plotly.graph_objs as go
from  ipywidgets import interact, interactive, fixed, interact_manual, Checkbox
import plotly.express as px
import warnings
import boto3
warnings.filterwarnings('ignore')

/Users/Pauline/.pyenv/versions/3.8.0/envs/mda4/lib/python3.8/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


In [3]:
s3 = boto3.client('s3')

# Data setup

The data (i.e., the CO2 per capita, ghg per capita, methane per capita, NO2 per capita columns) is only complete between ranges 1990 and 2016, so we'll have to remove the rest.

In [4]:
emissions0 = pd.read_csv('s3://s3grouppanama/Data/emissions.csv')
emissions1 = emissions0[emissions0['year'] <= 2016]
emissions2 = emissions1[emissions1['year'] >= 1990]
print(len(emissions2))
emissions2.index[pd.isnull(emissions2['co2_per_capita']) == True]

6089


Int64Index([ 4962,  4963,  4964,  4965,  4966,  4967,  4968,  4969,  4970,
             4971,
            ...
            23168, 23169, 23170, 23171, 23172, 23173, 23174, 23175, 23176,
            23177],
           dtype='int64', length=168)

Seems there's still missing data. Indeed, some countries did not report data at all. 

In [ ]:
emissions3 = emissions2.dropna( how='any',
                    subset=['co2_per_capita', 'ghg_per_capita', 'methane_per_capita', 'nitrous_oxide_per_capita'])
display(emissions3)

Now for the energy dataset. This dataset has information between 1990 and 2019, but we're trimming it down to 2016 to match the emissions dataset. Important: <ol>
<li>Renewables_share = solarshare+windshare+hydroshare+otherrenewables </li>
<li>fossilshare = coalshare + gasshare + oilshare </li>
<li>lowcarbonshare = renewablesshare + nuclearshare </li> </ol>

In [ ]:
energy0 = pd.read_csv('s3://s3grouppanama/Data/energy.csv')
energy1 = energy0[energy0['year'] <= 2016]
energy2 = energy1[energy1['year'] >= 1990]

However, we can even see from just the display that this one also has countries without any data, so we'll do the same as before but now for the relevant variables here. In this case, all variables are relevant so we have to remove all countries with any missing values.

In [ ]:
energy3 = energy2.dropna()
display(energy3)

Now we make the main dataset.

In [ ]:
data0 = pd.merge(emissions3, energy3,  how='inner', on=['country','year'])
data0 = data0[data0['country']!="World"]
display(data0)

Strangely, certain countries have negative ghg values. Let's take a look at what countries these are.

In [ ]:
data0.loc[data0["total_ghg"]<=0, ["nitrous_oxide","co2","methane", "total_ghg", "country", "year"]]

So Chile, Latvia, and Romania have negative ghg emissions for certain years. Given that their emissions of CO2, methane, and NO2/NO3 are non-negative. We cannot explain this anomaly, so we will leave out these countries.

In [ ]:
data0 = data0[data0.country != "Chile"]
data0 = data0[data0.country != "Latvia"]
data0 = data0[data0.country != "Romania"]
display(data0)
print("Final number of countries in the dataset:",len(set(data0.country)))

#  Let's play around with the data to get a feel of it

This world map shows for which countries we have information and for which countries we don't. The data is pretty complete but it's clear we can't make inferences about Africa and certain places in Asia. We can also get an idea of the energy production setup of the countries as well as how much greenhouse gasses (per capita) they produce.

In [ ]:
country = pd.read_excel('s3://s3grouppanama/Data/country2.xls')
data1 = pd.merge(data0, country, how='inner', on=['country'])
variables = {'renewables_share_energy': 'Renewables share of energy',
             'nuclear_share_energy': 'Nuclear share of energy',
             'fossil_share_energy': 'fossil share of energy',
            'co2_per_capita' : 'CO2 per capita'}
year = list(range(1990,2016))
v_key= list(["renewables_share_energy","fossil_share_energy","nuclear_share_energy","co2_per_capita"])
def f(year, v_key):
    data=data1[data1['year']==year]
    data1map=dict(type='choropleth',
         locations=data['country'],
         locationmode= 'country names',
         colorscale='sunset',
         text=data['country'],
         z=data[v_key],
         marker=dict(line=dict(color='rgb(300,300,300)', width=1)),
         colorbar={'title': variables[v_key]})  
    layout=dict(geo={'scope' : 'world'}) 
    choromap=go.Figure(data=[data1map], layout=layout) 
    choromap.show() 
interactive(f, year=year, v_key = v_key)

This line chart displays the evolution over time of the energy shares by country. This evolution is quite stable, suggesting that if we find an effect, it likely will not be large. Also, most countries run primarily on fossil fuels, which isn't really a surprise. Exceptions such as Norway and France exist of course.

In [ ]:
country = sorted(list(set(data0['country'])))

df = ['nuclear_share_energy','renewables_share_energy','fossil_share_energy']

m_data0 = pd.melt(data0, id_vars = ["country", "year"], value_vars = df)

def f(country):
    plt.figure(figsize=(10,5))

    sns.lineplot(x='year',y='value', data=m_data0[m_data0['country']==country],legend='auto', hue = "variable")
    plt.ylim(-5, 105)
    plt.xlim(1989, None) 
interactive(f,country=country)


These line charts describe the emitted greenhouse gases. You may notice some strange phenomena where "greenhouse gases per capita" (ghg_per_capita) are lower than "CO2 per capita" (co2_per_capita). This, like the negative ghg-values we removed, is something we cannot explain and is likely due to the way ghg_per_capita was calculated (but we do not have that information). The trends fluctuate more than the energy shares. From the parallel coordinates plot, we can guess that ghg_per_capita will likely be correlated to co2_per_capita.

In [ ]:
country = sorted(list(set(data0['country'])))

df = ['ghg_per_capita','co2_per_capita','methane_per_capita','nitrous_oxide_per_capita']

m_data0 = pd.melt(data0, id_vars = ["country", "year"], value_vars = df)

def f(country):
    plt.figure(figsize=(10,5))

    sns.lineplot(x='year',y='value', data=m_data0[m_data0['country']==country],legend='auto', hue = "variable")
    plt.ylim(-1, 40)
    plt.xlim(1989, None) 
interactive(f,country=country)


# Exploring the outcome variables

Let's analyse the distribution of our outcome variables first. <br>
There are four candidates: <ol>
<li>ghg_per_capita: Total greenhouse gas production per capita for a given year, expressed in tonnes of CO2 equivalent greenhouse effect.Interestingly, this quantity is not the sum of the other three. It is systematically higher than any of them, but not higher than their sum. </li>
<li>co2_per_capita: Total CO2 production per capita for a given year, expressed in tonnes. This is by far the largest of the three in magnitude. </li>
<li>methane_per_capita: Total methane (CH4) production per capita for a given year, expressed in tonnes of CO2 equivalent greenhouse effect. </li>
<li>nitrous_oxide_per_capita: Total methane (NO2/NO3) production per capita for a given year, expressed in tonnes of CO2 equivalent greenhouse effect. </li></ol>

All four have similar right-skewed distributions. 

In [ ]:
plt.figure(figsize=(20,5))
plt.subplot(1, 2, 1)
f1 = sns.histplot(data0.ghg_per_capita)
f1.set(xlabel='Greenhouse gas production per capita')
plt.subplot(1, 2, 2)
f2=sns.histplot(data0.co2_per_capita)
f2.set(xlabel='CO2 production per capita')

plt.figure(figsize=(20,5))
plt.subplot(1, 2, 1)
f3=sns.histplot(data0.methane_per_capita)
f3.set(xlabel='Methane (CH4) production per capita')
plt.subplot(1, 2, 2)
f4=sns.histplot(data0.nitrous_oxide_per_capita)
f4.set(xlabel='Nitrous oxide (NO2/NO3) production per capita'); 


Let's take a look at the correlation structure of our outcome variables. As expected, they're positively correlated. CO2 is highly correlated with ghg (r = 0.88), suggesting a lot of the emitted greenhouse gasses are CO2 (which makes sense looking at the magnitudes of the emissions). This trend is also visible in the parallel coordinates plot (where one line represents one observation (year), and the lines are coloured by country). For this analysis, we will only be regressing ghg_per_capita. <br>

In [ ]:
plt.figure(figsize=(10,7))
print(data0[['ghg_per_capita','co2_per_capita','methane_per_capita', 'nitrous_oxide_per_capita']].corr())
year = range(1992, 2017)
country = list(set(data0['country']))
p = (data0[(data0['country'].isin(country))] .loc[:, ['country', 'co2_per_capita','ghg_per_capita','methane_per_capita', 'nitrous_oxide_per_capita']])
parallel_coordinates(p, 'country')
plt.gca().legend_.remove()

<p> Although we're not regressing against time, it is worth it taking a look at the evolution of the emissions between 1990 and 2016. (Note that this cell can take a good minute to run!). </p>
<p>The overall evolution of greenhouse gas per capita (ghg_per_capita) is chaotic, but it seems to be increasing to a peak in 2004, after which it goes down again. This evolution can be seen in a lot (although not all) of the individual countries as well, such as Canada, Belgium, Belarus, and Egypt. Other countries show a wide variety of trends.</p>


In [ ]:
sns.catplot(x="year", y="ghg_per_capita", data=data0, kind = "swarm", s=1.4)
plt.xticks(rotation=90)
sns.catplot(x="year", y="ghg_per_capita", col="country", col_wrap=4, sharey=False, data=data0, kind = "swarm");

# Exploring the regressor variables

The regressors all represent a percentage share of the total energy production. <br> <ul>
<li>Fossil is the sum of coal, gas, and oil. </li>
<li>Renewables is the sum of wind, hydro, solar, biofuel, and other renewables. </li>
    <li>Low carbon is the sum of renewables and nuclear. </li></ul>

To investigate the correlations between the regressors, we'll use a heatmap because there's a lot of them. <br>
We see some expected patterns in the data. Fossil is perfectly correlated to low-carbon, which makes sense since low-carbon was defined as everything that wasn't fossils. <br>

In [ ]:
regressors = ["low_carbon_share_energy","renewables_share_energy","fossil_share_energy","biofuel_share_energy","coal_share_energy","gas_share_energy","hydro_share_energy","nuclear_share_energy","oil_share_energy","other_renewables_share_energy","solar_share_energy","wind_share_energy"]


fig, ax = plt.subplots(figsize=(10, 8))
mask = np.triu(np.ones_like(data0[regressors].corr(), dtype=np.bool))
mask = mask[1:, :-1]
corr = data0[regressors].corr().iloc[1:,:-1].copy()
cmap = sns.diverging_palette(0, 230, 90, 60, as_cmap=True)
sns.heatmap(corr, mask=mask, annot=True, fmt=".2f", 
           linewidths=5, cmap=cmap, vmin=-1, vmax=1, 
           cbar_kws={"shrink": .8}, square=True)
yticks = [i.upper() for i in corr.index]
xticks = [i.upper() for i in corr.columns]
plt.yticks(plt.yticks()[0], labels=yticks, rotation=0)
plt.xticks(plt.xticks()[0], labels=xticks)
title = 'CORRELATION MATRIX\n REGRESSORS\n'
plt.title(title, loc='left', fontsize=10)
plt.show()



<p>If we look at the variances of the regressors, we notice that there is a substantial difference in magnitude. Using them all as regressors would make a lot of them redundant (and probably overfit the model to death). So, we should pick a few. </p>
<p>For the first model, we will simply use renewables, fossil, and nuclear, because these three cover the rest and are intuitive categories. Later, we will run a PCA and see if maybe we can get a new set of variables to fit a better model. </p>

In [ ]:
print("Variances:\n",data0[regressors].var())

# Exploring the relationship between the regressors and the outcome

Now, we're not regressing against time but against the different kinds of energy production, so we should take a look at how those compare to the emissions. Some observations: <ol>
<li>Most observations lie on the higher end of fossil fuel shares. That's not too unexpected given that the main source of electricity in the world is still fossil fuels. It seems some countries even reach 100%. The overall trend seems to be a slight linear increase, but with a sudden exponential growth at high values of fossil share. There are also some observations around fossil_share ~ 70 with unusually high ghg emissions that somewhat break the trend. </li> <br>
<li>For the renewables share we can make almost identical observations but flipped. The trend looks like a linear decrease with deviations at around 0 and 30 (which are the complements of 100 and 70 from the fossil share plot). </li> <br>
<li>The nuclear shares plot has a somewhat similar shape to the renewable one, but with a weaker slope. It also seems less dense. This is actually because there is a large concentration of countries with zero nuclear energy. In fact, more than half of all countries in the list do not have nuclear reactors generating power.</li></ol>

In [ ]:
plt.figure(figsize=(20,5))
sns.lmplot(x = "fossil_share_energy", y = "ghg_per_capita", data = data0)
sns.lmplot(x = "renewables_share_energy", y = "ghg_per_capita", data = data0)
sns.lmplot(x = "nuclear_share_energy", y = "ghg_per_capita", data = data0);

Let's investigate the nuclear power issue further. In fact, 59.5% of countries have never had nuclear power (at least as of 2016), and 62.0% of them have not had nuclear power at some point (between 1990 and 2016).

In [ ]:
datasum = data0[['country','ghg_per_capita','nuclear_share_energy','renewables_share_energy','fossil_share_energy']].groupby(['country']).sum()
datasum['country']=datasum.index
datasum["total_energy"]=datasum.nuclear_share_energy+datasum.renewables_share_energy+datasum.fossil_share_energy
print("Percentage of countries that never had nuclear power:" , len(datasum[datasum['nuclear_share_energy']==0])/len(datasum))
print("Percentage of countries that did not consistently have nuclear power between 1990 and 2016: " , len(data0[data0['nuclear_share_energy']==0])/len(data0))


Taking a look at the distributions of our three regressors, we see that they are all heavily skewed, which is in line with our observations in the previous plots. However, nuclear power is, as expected, even worse.

In [ ]:
sns.distplot(data0.nuclear_share_energy)
plt.figure()
sns.distplot(data0.fossil_share_energy)
plt.figure()
sns.distplot(data0.renewables_share_energy);

Taking a look at the distribution of nuclear shares when we ignore the countries without nuclear power, we see it is still heavily skewed towards the lower end, with a median of ~10.

In [ ]:
sns.distplot(data0.nuclear_share_energy[data0.nuclear_share_energy !=0])
plt.figure()
print("median:" , stats.median(data0.nuclear_share_energy[data0.nuclear_share_energy !=0]))

All these reasons would make regressing against nuclear_share_energy difficult, as the model would forcefully try to fit the odd data. However, removing all countries without nuclear power would leave us with a 60% reduced dataset, which is undesirable. Therefore, a good compromise is to categorise nuclear_share_energy into three groups: <ol>
<li>No nuclear power </li>
<li>Nuclear power representing 0 to 10% (the median) of total output </li>
<li>Nuclear power representing >10% of total output. </li> </ol>
However, this can fluctuate: a country might cross this threshold at some point. To avoid overfitting the curvature of each country, we will categorize based on *total* energy production over the entire time span 1990-2016. So a country will end up in the higher category only if >10% of their total energy production was nuclear.

In [ ]:
for i in set(datasum['country']):
    if datasum.loc[i,'nuclear_share_energy'] == 0:
        for j in data0.index[data0["country"]==i]:
            data0.loc[j,'nuclear'] = "No"
    elif 0<datasum.loc[i,'nuclear_share_energy']/datasum.loc[i,'total_energy']<0.1:
         for j in data0.index[data0["country"]==i]:
            data0.loc[j,'nuclear'] = "<10%"
    else:    
        for j in data0.index[data0["country"]==i]:
            data0.loc[j,'nuclear'] = ">10%"

<p>So now we have our regressors. However, a regular linear regression wouldn't cut it, because we would be violating one of the key assumptions: the independence of measurements. Indeed, two observations in a linear regression need to be independent of each other. Violating this assumption may lead to a substantial loss of power. </p>
<p>In our dataset, it would intuitively make sense for observations from the same country to be correlated. This is confirmed in the following graphs. These are the same as the regressor vs ghg graphs from before, but now dots corresponding to the same country are connected. These graphs reveal a large amount of autocorrelation on both axes: countries that start low, tend to stay low, and vice versa.</p>

In [ ]:
fig = plt.figure()
ax = plt.subplot(111)
fig.suptitle('Individual profiles of country co2 emissions vs share of renewable energy')
plt.xlabel('renewables_share_energy')
plt.ylabel('ghg_per_capita')
plt.ylim([0,40])
for i in set(data0['country']):
    country_i = data0.loc[data0['country'] == i]
    x=country_i['renewables_share_energy']
    y=country_i['ghg_per_capita']
    order = np.argsort(x)
    xs = np.array(x)[order]
    ys = np.array(y)[order]
    ax.plot(xs, ys)
plt.show()
fig = plt.figure()
ax = plt.subplot(111)
fig.suptitle('Individual profiles of country co2 emissions vs share of nuclear energy')
plt.xlabel('nuclear_share_energy')
plt.ylabel('ghg_per_capita')
plt.ylim([0,40])
for i in set(data0['country']):
    country_i = data0.loc[data0['country'] == i]
    x=country_i['nuclear_share_energy']
    y=country_i['ghg_per_capita']
    order = np.argsort(x)
    xs = np.array(x)[order]
    ys = np.array(y)[order]
    ax.plot(xs, ys)
plt.show()
fig = plt.figure()
ax = plt.subplot(111)
fig.suptitle('Individual profiles of country co2 emissions vs share of fossil energy')
plt.xlabel('fossil_share_energy')
plt.ylabel('ghg_per_capita')
plt.ylim([0,40])
for i in set(data0['country']):
    country_i = data0.loc[data0['country'] == i]
    x=country_i['fossil_share_energy']
    y=country_i['ghg_per_capita']
    order = np.argsort(x)
    xs = np.array(x)[order]
    ys = np.array(y)[order]
    ax.plot(xs, ys)
plt.show()

This assumption being violated is a pretty big deal. Luckily, mixed models can save us. By taking into account the between-country variability with an additional random term we can get some of our lost power back and reveal the hidden trends.

# The first model

So we are now running the mixed model. <br>
We are regressing total greenhouse gas production against share of fossil energy and share of renewable energy, while correcting the intercept for nuclear power capabilities. <br>
We also add allow for random intercepts and random slope effects for each country.<br>

<p><b>Conclusions from this model </b></p>
<p>Both the share of fossil energy and renewable energy comes out significant (although the latter one is borderline). As expected, a larger share of fossil fuels increases ghg emissions, and a larger share of renewables decreases ghg emissions. The magnitude of these two effects is more or less equal but opposite. The interpretation of this coefficient would be "for each unit increase (1%) in share of fossil fuel energy production, the output of total ghg increases by about 0.202 tonnes of CO2-equivalent per capita, controlling for nuclear energy and renewable energy". </p>
<p>The intercept does not significantly differ based on countries' nuclear capabilities. The intercept is essentially the ghg output for a country with zero fossil or renewable energies, and the model states that the ghg emissions for such a country do not depend on whether or not it has nuclear power generators. However, the intercept is seldom informative because we are extrapolating outside our (already messy) dataset. </p>

In [ ]:
md = smf.mixedlm("ghg_per_capita ~ fossil_share_energy+renewables_share_energy+C(nuclear,Treatment(reference='No'))", 
                 data0, 
                 groups=data0["country"],
                re_formula="~fossil_share_energy+renewables_share_energy")
mdf=md.fit()
print(mdf.summary())
print("residual variance:" , stats.variance(mdf.resid.values))

We can also use this model to make predictions about a hypothetical country's ghg output given its energy production capabilities. Note that the limitations of our model are apparent here: for any predictions that have a share of fossil generated energy of less than 40%, the predicted ghg output per capita is negative. It's technically not extrapolation, because the lowest value for fossil_share_energy in our dataset is 17%, but only ~3.5% of observations had values under 40%. Still, we should be careful about making inferences like this without taking the random effects of the specific country into account.

In [ ]:
Fossil_percent = list(range(round(data0["fossil_share_energy"].min()),round(data0["fossil_share_energy"].max())))
Renewables_percent = list(range(round(data0["renewables_share_energy"].min()),round(data0["renewables_share_energy"].max())))
Nuclear_percent = list(['No','<10%','>10%'])
def f(Fossil_percent, Renewables_percent,Nuclear_percent):
    ex = pd.DataFrame({'fossil_share_energy': [Fossil_percent], 'renewables_share_energy': [Renewables_percent], 'nuclear': [Nuclear_percent]})
    print(mdf.predict(exog=ex))
interactive(f, Fossil_percent=Fossil_percent, Renewables_percent = Renewables_percent, Nuclear_percent=Nuclear_percent)

<p>Assessing model fit for a mixed model is not as straightforward as for a non-mixed model, but a good way to start is by comparing the variance terms. </p>
<p>The residual variance has been printed underneath the output, and is equal to ~3. Meanwhile, the between-country variance is ~596. This implies that the vast majority of variability in our dataset is country-specific noise, rather than residual noise. This shows the importance of accounting for autocorrelation in such a dataset. However, it also suggests that the effects we detected are only a fraction of what is 'really going on'. This also makes sense. A countries' GHG emissions depend on much more than just its energy production. </p>
<p>We should also check the assumptions of this model. Residual plots are useful for this.
Residuals vs Predicted: The size of the residuals is pretty constant over the whole spectrum of predicted values. This implies that a linear mixed model fits the data and that there is no need to use generalized mixed models. </p>
<p> <i>Residuals vs regressor variables: </i>These plots should ideally be constant, yet we see a funnel shape in both of them. Deviations in these plots can often be an indication of a missing higher-order term. However, there is no clear polynomial structure in these plots. We suspect the funnel shape is caused by the serious imbalance in the data (the majority of countries have low renewables, high fossils).</p>
<p> <i>Residuals vs random effects: </i>An assumption of any linear model is the normality of the random effects (with mean zero). The residual random term has a nice looking gaussian histogram with a variance of 2.94 (residual variance). The group, fossil_share, and renewables_share distributions are skewed. Linear regressions are usually pretty robust against violations of normality, though. The variance of the group effect is 595.79. The variance of the random fossil term is 0.111. Taking into account that the estimated mean slope for the fossil effect was 0.202, shows that a variance of 0.111 is substantial. This means that the effect of reducing/increasing fossil fuel energy production depends substantially by country. The same reasoning can be made with the renewables error term that has a variance 0.230.</p>


In [ ]:
modelfit = pd.DataFrame()
modelfit["residuals"] = mdf.resid.values
modelfit["predicted"] = mdf.fittedvalues
modelfit["fossils"] = data0["fossil_share_energy"]
modelfit["renewables"] = data0["renewables_share_energy"]

sns.lmplot(x="predicted", y="residuals", data=modelfit)
plt.figure()
sns.lmplot(x="fossils", y="residuals", data=modelfit)
plt.figure()
sns.lmplot(x="renewables", y="residuals", data=modelfit)
plt.figure()
sns.distplot(modelfit["residuals"])
randeffects = pd.DataFrame.from_dict(mdf.random_effects).transpose()
plt.figure()
sns.distplot(randeffects["Group"])
plt.figure()
sns.distplot(randeffects["fossil_share_energy"])
plt.figure()
sns.distplot(randeffects["renewables_share_energy"]);

# The PCA model

We started this analysis with a large number of covariates, but we ended up running a model with only three. That is of course because we grouped them together in three categories that made sense to us. However, these categories are constructed based on real-life interpretation of these energies, not based on what the data tells us. So instead let's run a PCA to see how a purely data-driven technique would allocate the variability of our regressors if it was constrained to two or three components. We'll now be leaving out the grouped variables (fossil, renewables, low carbon). Let's first take a look again at the variances of each variable. Since PCA as a method relies on grouping together variance, these large differences in magnitude will be problematic. Variables like 'solar_share_energy' and 'biofuel_share_energy' are unlikely to have any meaningful loadings.

In [ ]:
datapca = data0[[
"biofuel_share_energy","coal_share_energy","gas_share_energy","hydro_share_energy","oil_share_energy","other_renewables_share_energy","solar_share_energy","wind_share_energy"
]]
print("Variances:\n" , datapca.var())

And indeed, the PCA confirms this. We see that more than 90% of the variability in the dataset can be explained by the first two components, whose weights are mostly gas, coal, and oil. Interestingly though, oil seems to be negatively correlated to coal (both loading strongly but oppositely on the same component).

In [ ]:
values = datapca.values
pca = PCA()
pca.fit(values)
fig,ax = plt.subplots(1,1,figsize=(15,5))
ax.plot(np.arange(0,len(datapca.columns)),100*np.cumsum(pca.explained_variance_ratio_));
ax.set_ylabel('% Variance Explained')
ax.set_xlabel('EigenVectors considered');
pca.components_.T
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2','PC3','PC4','PC5','PC6','PC7','PC8'], index=[
"biofuel_share_energy","coal_share_energy","gas_share_energy","hydro_share_energy","oil_share_energy","other_renewables_share_energy","solar_share_energy","wind_share_energy"
])
display(loadings)

Index= ["biofuel_share_energy","coal_share_energy","gas_share_energy","hydro_share_energy","oil_share_energy","other_renewables_share_energy","solar_share_energy","wind_share_energy"]
Cols = ['PC1', 'PC2','PC3','PC4','PC5','PC6','PC7','PC8']
df = pd.DataFrame(loadings, index=Index, columns=Cols)
plt.figure()

sns.heatmap(df, annot=True)

<p>To counteract the dominance of the three fossil fuel energies, we're going to standardise all variables and run a PCA again. </p>
<p>The results are a mixed bag. On the one hand, all variables now get a chance to shine, on the other hand, that's because none of the components explain much variability at all. We could use like 5 or 6 of them, but that would defeat the point of dimension reduction. So, we're going to use the first three. Note that these only account for about 70% of the variability in our dataset, which means that right off the bat this model's conclusions might be biased. </p><ul>
<li>Component 1 seems to positively correlate with renewable energies (mostly hydro and other renewables). It is negatively correlated with gas. </li>
<li>Component 2 seems to be positively correlated with coal, solar, and wind. </li>
    <li>Component 3 seems to be positive for gas, negative for coal.</li></ul>

In [ ]:
df = [
"biofuel_share_energy","coal_share_energy","gas_share_energy","hydro_share_energy","oil_share_energy","other_renewables_share_energy","solar_share_energy","wind_share_energy"
]

for item in df :
    item_std = item+"_std"
    data0[item_std] = (data0[item] - data0[item].mean()) / data0[item].std()
datapca = data0[[
"biofuel_share_energy_std","coal_share_energy_std","gas_share_energy_std","hydro_share_energy_std","oil_share_energy_std","other_renewables_share_energy_std","solar_share_energy_std","wind_share_energy_std"
]]
values = datapca.values
pca = PCA()
pca.fit(values)
fig,ax = plt.subplots(1,1,figsize=(15,5))
ax.plot(np.arange(0,len(datapca.columns)),100*np.cumsum(pca.explained_variance_ratio_));
ax.set_ylabel('% Variance Explained')
ax.set_xlabel('EigenVectors considered');
pca.components_.T
loadings = pd.DataFrame(pca.components_.T, columns=['PC1', 'PC2','PC3','PC4','PC5','PC6','PC7','PC8'], index=[
"biofuel_share_energy","coal_share_energy","gas_share_energy","hydro_share_energy","oil_share_energy","other_renewables_share_energy","solar_share_energy","wind_share_energy"
])
display(loadings)

Index= ["biofuel_share_energy","coal_share_energy","gas_share_energy","hydro_share_energy","oil_share_energy","other_renewables_share_energy","solar_share_energy","wind_share_energy"]
Cols = ['PC1', 'PC2','PC3','PC4','PC5','PC6','PC7','PC8']
df = pd.DataFrame(loadings, index=Index, columns=Cols)
plt.figure()

sns.heatmap(df, annot=True)

So let's run a 3 component pca again and extract our new dataset of covariates:

In [ ]:
pca3 = PCA(n_components=3)
values = datapca.values
pca3.fit(values)
t = datapca.index
Z=pd.DataFrame(pca3.transform(datapca))
Z.columns = ['Renew_vs_Gas','CoalandWindandSolar_vs_Hydro', 'Gas_vs_Coal']
Z.index = data0.index
Z["country"] = data0.country
newdata = pd.concat([Z, data0[['ghg_per_capita','nuclear']]], axis=1)
display(newdata)

<p>And let's run the model. We are not running random slope effects anymore. Doing so prevents the model from converging. In mixed models, this is typically the result of overfitting. After all, having four random effects would make the covariance structure quite complex. Instead, we just run random intercepts based on country. </p>
<p>The conclusions regarding the intercept are the same as before: The nuclear capabilities of the country do not have an effect. All slopes are significant: <ol>
<li>Increasing gas/reducing renewables (biofuel, hydro, other renewables) increases ghg emissions. This is expected </li>
<li>Increases coal, wind, and solar/decreasing hydro increases ghg emissions. It's hard to tell if wind and solar legitimately contribute to ghg emissions or if the coal part of the principal component dominates. The conclusion for hydro makes sense. </li>
<li>Increasing gas/decreasing coal *decreases* ghg emissions. An interesting observation that is also in line with our knowledge. Natural gas burning is known to produce a lot fewer emissions compared to coal burning.</li></ol> </p> 

In [ ]:
mdpca = smf.mixedlm("ghg_per_capita ~ Renew_vs_Gas+CoalandWindandSolar_vs_Hydro+Gas_vs_Coal+C(nuclear,Treatment(reference='No'))", 
                 newdata, 
                 groups=newdata["country"])
mdfpca=mdpca.fit()
print(mdfpca.summary())
print("Residual variance:",stats.variance(mdfpca.resid.values))

We can again take a look at the model fit. The residual variance is nearly identical, being 3.6 (compared to 2.9 before). The between-country variance however is a lot lower than it was in the first model (54 compared to 596). So comparatively, this model captures a 'larger' part of the variability in its fixed effects. Although we must recall that only ~70% of the total variability in our covariates is accounted for. 
<p>Taking a look at the residual plots. The conclusions are pretty much the same as for the first model.</p>

In [ ]:
modelfitpca = pd.DataFrame()
modelfitpca["residuals"] = mdfpca.resid.values
modelfitpca["predicted"] = mdfpca.fittedvalues
modelfitpca["Renew_vs_Gas"] = Z["Renew_vs_Gas"]
modelfitpca["CoalandWindandSolar_vs_Hydro"] = Z["CoalandWindandSolar_vs_Hydro"]
modelfitpca["Gas_vs_Coal"] = Z["Gas_vs_Coal"]

sns.lmplot(x="predicted", y="residuals", data=modelfitpca)
plt.figure()
sns.lmplot(x="Renew_vs_Gas", y="residuals", data=modelfitpca)
plt.figure()
sns.lmplot(x="CoalandWindandSolar_vs_Hydro", y="residuals", data=modelfitpca)
plt.figure()
sns.lmplot(x="Gas_vs_Coal", y="residuals", data=modelfitpca)
plt.figure()
sns.distplot(modelfitpca["residuals"])
randeffectspca = pd.DataFrame.from_dict(mdfpca.random_effects).transpose()
plt.figure()
sns.distplot(randeffectspca["Group"]);

# Conclusions

Our analysis included the annual energy production composition of 74 countries as well as their annual per capita greenhouse gas emissions from 1990 to 2016. We recall that a "unit" of ghg is expressed in tonnes of CO2-equivalent (when considering the potential for greenhouse effect).
The conclusions of our analysis are the following: <ol>
<li>The variability between countries is much greater than the variability within countries. Suggesting that there are larger factors at play that impact a countries' ghg emissions, besides their energy production. </li> <br>
Still, some significant trends were found: <br>
<br><li>A higher share of renewable energies tends to lead to less ghg emissions, *on average*. In fact, for every 1% of the energy production that is switched over to renewables, the country will produce about 0.199 units of ghg per capita per year less, <i>on average</i>, keeping fossil share energy and nuclear energy constant. </li>
<br><li>Similarly, for every 1% of the energy production that is switched over to fossil energy, the country will produce about 0.202 units of ghg per capita per year more, <i>on average</i>, controlling for renewable energy and nuclear energy. </li>
<br><li>The share of energy that is generated by nuclear facilities does not seem to have an impact on the units of ghg per capita per year produced, <i>on average</i>. </li>
<br><li>If using fossil fuels is inevitable, using natural gas is preferable to using coal (if only ghg emissions are considered). </li><br>
So can we use this to make predictions? <br>
<br><li>Making country-specific predictions is difficult, because it requires accurate estimation of the random effects, which can be difficult given the noise in the data. This model is more valuable for comparing the average effect of different energy production types on ghg emissions.</li></ol>